# NUMBA experiements

Create a function that can be easily adapted to calculate the heat transport from MITgmc

In [ ]:
from netCDF4 import Dataset
import os

#use OFES to play around
url = 'http://apdrc.soest.hawaii.edu:80/dods/public_ofes/OfES/ncep_0.1_global_mmean/uvel'
dataset = Dataset(url, mode = 'r')

In [ ]:
def transport(dataset, year, yy):
    #Inputs: dataset, year and latitude
    
    from netCDF4 import Dataset
    
    print(year)
    uvel = dataset.variables['uvel'][year, :, yy-2:yy+2, :]
    uvel[uvel == -9999] = 0
    
    u_cum = 0
    for i in range(uvel.shape[3]): #lon
        for j in range(uvel.shape[1]): #lat
            for n in range(uvel.shape[2]): #depth
                u_cum = u_cum + uvel[n,j,i]#later, here multiply by temperature a thermal coeff
                
    u_cum.tofile(year+'.bin') #for testing
    
    return u_cum

In [ ]:
%%time
transport(dataset, 2000, 10)

## Try NUMBA

In [ ]:
from numba import jit

In [ ]:
@jit(nopython=True)
def calc_cum(a):
    u_cum = 0
    for i in range(uvel.shape[3]): #lon
        for j in range(uvel.shape[1]): #lat
            for n in range(uvel.shape[2]): #depth
                u_cum = u_cum + uvel[n,j,i]#later, here multiply by temperature a thermal coeff
    return u_cum

def transport_numba(dataset, year, yy):
    #Inputs: dataset, year and latitude
    
    from netCDF4 import Dataset
    
    print(year)
    uvel = dataset.variables['uvel'][year, :, yy-2:yy+2, :]
    uvel[uvel == -9999] = 0
    
    u_cum = calc_cum(a)
    
    f = np.array(u_cum)
    f.tofile(year+'.bin')
    
    return f

In [ ]:
%%time
transport_numba(dataset, 2000, 10)

This was using only one processor.

Let's try now with two!

**NOTA BENE** multuprocessing is incompatible with jupyter atm

In [ ]:
import multiprocessing

%%time
pool = multiprocessing.Pool(processes=2)
r = pool.map(transport_numba, dataset, 2000, 10)
pool.close()

On my laptop, this runs a bit more than 2 times faster.

**NOTA BENE** Specifying the output type speeds up the code a tiny bit more.

In [ ]:
@jit(nopython=True,'f8(f4[:,:,:])')